<a href="https://colab.research.google.com/github/EziamaUgonna/Bayesian_analysis-/blob/master/Copy_of_Dissertation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade -q gspread
from tensorboardcolab import *
import shutil
#clean out the directory
shutil.rmtree('./Graph', ignore_errors=True)
os.mkdir('./Graph')
#tf.reset_default_graph()
#will start the tunneling and will print out a link:
tbc=TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://ea464da4.ngrok.io


In [2]:
!pip show tensorflow

Name: tensorflow
Version: 1.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: keras-preprocessing, termcolor, protobuf, numpy, keras-applications, absl-py, gast, tensorboard, tensorflow-estimator, grpcio, wheel, astor, six
Required-by: stable-baselines, magenta, fancyimpute


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals 

In [4]:
# Dependencies
import os 
import warnings 
#from absl import flags 
import matplotlib 
import numpy as np 
import tensorflow as tf 
import tensorflow_probability as tfp
import math
import pandas as pd
tfd = tfp.distributions
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials
#import python_utils



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
# clear graph (if any) before running 
tf.reset_default_graph()

In [6]:
####Delete all flags before declare#####

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags
FLAGS = tf.app.flags.FLAGS
flags.DEFINE_float("learning_rate", default = 0.0001, help = "Initial learning rate.")
flags.DEFINE_integer("epochs", default = 700, help = "Number of epochs to train for")
flags.DEFINE_integer("batch_size", default =128, help = "Batch size.")
flags.DEFINE_integer("eval_freq", default = 400, help =" Frequency at which to validate the model.")
flags.DEFINE_float("kernel_posterior_scale_mean", default = -0.9, help = "Initial kernel posterior mean of the scale (log var) for q(w)")
flags.DEFINE_float("kernel_posterior_scale_constraint", default = 0.2, help = "Posterior kernel constraint for the scale (log var) for q(w)")
flags.DEFINE_float("kl_annealing", default = 50, help = "Epochs to anneal the KL term (anneals from 0 to 1)")
flags.DEFINE_integer("num_hidden_layers", default = 4, help = "Number of hidden layers")
flags.DEFINE_integer("num_monte_carlo",

                     default=50,

                     help="Network draws to compute predictive probabilities.")
tf.app.flags.DEFINE_string('f', '', 'kernel')
#initialize flags 
#FLAGS = flags.FLAGS
print(FLAGS.learning_rate)
print(FLAGS.epochs)
print(FLAGS.num_monte_carlo)

0.0001
700
50


In [0]:
def build_input_pipeline(X_train,X_test,y_train, y_test, batch_size, valid_size):
  #Build an iterator over training batches 
  training_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  #Shuffle the dataset (note shuffle argument much larger than training size)
  # and form batches of size batch_size
  training_batches = training_dataset.shuffle(20000, reshuffle_each_iteration =True).repeat().batch(batch_size)
  training_iterator = tf.data.make_one_shot_iterator(training_batches)
  
  #Building iterator over the heldout set with batch_size = heldout_size,
  # i.e., return the entire heldout set as a constant.
  heldout_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
  heldout_batches = heldout_dataset.repeat().batch(valid_size)
  heldout_iterator = tf.data.make_one_shot_iterator(heldout_batches)
  
  #Combine these into a feasible iterator that can switch between training 
  # and validation inputs.
  # Here should be minibatch increment be defined 
  handle = tf.placeholder(tf.string, shape = [])
  feedable_iterator = tf.data.Iterator.from_string_handle(handle, training_batches.output_types, training_batches.output_shapes)
  features_final, labels_final = feedable_iterator.get_next()
  
  return features_final, labels_final, handle, training_iterator, heldout_iterator


In [8]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
!ls "/content/gdrive/My Drive/"

 acc_1.csv   Bayesian_log_trial_.ipynb	 elvin.csv	   Untitled1.ipynb
 acc_2.csv   BNN_Work.ipynb		 kernel.ipynb	   work2.csv
 acc_3.csv  'Colab Notebooks'		 preds_2.npy	   work_main
 acc_4.csv   data1.csv			 preds_3.npy
 acc_5.csv   data23.csv			 Untitled0.ipynb


In [10]:
# Read in the dataset
df = pd.read_csv('/content/gdrive/My Drive/work2.csv').astype(np.float32)
change = df.query('Speed>0').sample(frac = .2).index
df.loc[change, 'Speed'] = 0
df.loc[change, 'Class'] = 0
df.to_csv('work2.csv', header = True, index =False)
df.shape

(1048575, 20)

In [0]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state =1)

#reshape y-data to become column vector 
y_train = np.reshape(y_train, [-1,1])
y_test = np.reshape(y_test, [-1,1])

In [0]:
# Standardize the dataset 
scalar_x_train = StandardScaler().fit(X_train)
scalar_x_test = StandardScaler().fit(X_test)
X_train = scalar_x_train.transform(X_train)
X_test = scalar_x_test.transform(X_test)

In [0]:
def main(argv):
  (features_final, labels_final, handle, training_iterator, heldout_iterator) = build_input_pipeline(X_train,X_test, y_train,y_test, FLAGS.batch_size, 500)
  
  #features, labels = build_input_pipeline(X,y, FLAGS.batch_size)
  #Building Bayesian Logistic 
  with tf.name_scope("logistic_regression", values=[features_final]):
    layer = tfp.layers.DenseFlipout(
        units=1,
        activation= tf.nn.relu,
        kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
        bias_posterior_fn=tfp.layers.default_mean_field_normal_fn())
    logits = layer(features_final)
   # logits = layer(features)
    labels_distribution = tfd.Bernoulli(logits=logits)
 #Compute the -ELBO as the loss, averaged over the batch size 
    t = tf.Variable(0.0)
    kl_regularizer =  t / (FLAGS.kl_annealing * len(X_train) / FLAGS.batch_size)
  
    #Compute the -ELBO as the loss. The kl term is annealed from 1 to 1 over 
    # the epochs specified by the kl_annealing flag.
    #log_likelihood = labels_distribution.log_prob(labels_final)
    #log_likelihood = labels_distribution.log_prob(labels)
   #neg_log_likelihood = tf.reduce_mean(tf.squared_difference(logits,labels_final))
    neg_log_likelihood = -tf.reduce_mean(input_tensor = labels_distribution.log_prob(labels_final))
    #neg_log_likelihood = -tf.reduce_mean(input_tensor = log_likelihood)
    
    #kl = sum(layer.losses)/len(X_train) * tf.minimum(1.0, kl_regularizer)
    kl = tf.cast(sum(layer.losses)/len(X_train), tf.float32) * tf.minimum(1.0, kl_regularizer)
    elbo_loss = neg_log_likelihood + kl
   
  """neg_log_likelihood = -tf.reduce_mean(labels_distribution.log_prob(features))
  kl = sum(layer.losses)/len(X_train)
  elbo_loss = neg_log_likelihood + kl """
  #Build metrics for evaluation. Predictions are formed from single forward 
  #pass of the probabilistic layers. They are cheap but noise predictions
  #predictions = tf.argmax(input = logits, axis=1)
  #accuracy, accuracy_update_op = tf.metrics.accuracy(labels= labels_final,  predictions = predictions)
  predictions = tf.cast(logits>0, dtype = tf.float32)
  
  
  with tf.name_scope("train"):
    train_accuracy, train_accuracy_update_op = tf.metrics.accuracy(labels= labels_final,  predictions = predictions)
    optimizer =tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)
    train_op = optimizer.minimize(elbo_loss)
    update_step_op = tf.assign(t, t+1)
    #init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
  
  with tf.name_scope("valid"):
    valid_accuracy, validation_accuracy_update_op = tf.metrics.accuracy(labels = labels_final,predictions = predictions)
    #TP, TN, FP, FN
    TP = tf.count_nonzero(predictions * labels_final)
    TN = tf.count_nonzero((predictions-1)*(labels_final-1))
    FP = tf.count_nonzero(predictions*(labels_final-1))
    FN = tf.count_nonzero((predictions-1)* labels_final)
    #precision, recall, f1
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * precision * recall / (precision + recall)
    tpr = TP/(TP+FN)
    fpr = FP/(TP+FN)
   
    init_op = tf.group(tf.global_variables_initializer(), 
                       tf.local_variables_initializer())
    stream_vars_valid = [v for v in tf.local_variables() if "valid" in v.name]
    reset_valid_op = tf.variables_initializer(stream_vars_valid)
    
    
    
  with tf.Session() as sess:
    
    sess.run(init_op)
    #Run the training looop
    # Fit the model to data.
    train_handle = sess.run(training_iterator.string_handle())
    heldout_handle = sess.run(heldout_iterator.string_handle())
    training_steps = int(round(FLAGS.epochs * (len(X_train) / FLAGS.batch_size)))
    for step in range(training_steps):
      _ = sess.run([train_op,train_accuracy_update_op, update_step_op],feed_dict={handle: train_handle})
      
      # Manually print the frequency 
      if step % 100 == 0:
        loss_value, accuracy_value, kl_value = sess.run([elbo_loss, train_accuracy, kl], feed_dict= {handle:train_handle})
        print("Step:{:>3d} loss : {:.3f} KL: {:.3f}" .format(step , loss_value, accuracy_value, kl_value))
        
      if (step +1) % FLAGS.eval_freq ==0: 
        # Compute log prob of heldout set by averaging draws from the model:
        # p(heldout | train) = int_model p(heldout|model) p(model|train) ~= 1/n * sum_{i=1}^n p(heldout | model_i)
        # where model_i is a draw from the posterior 
        #p(model|train)
        probs = np.asarray([sess.run((labels_distribution.probs), 
                                     feed_dict ={handle: heldout_handle})
      
                            for _ in range(FLAGS.num_monte_carlo)])
     
        mean_probs = np.mean(probs, axis =0).astype(np.int32)
        print(mean_probs.dtype)
        
        _, label_vals = sess.run((features_final, labels_final), feed_dict = {handle: heldout_handle})
        label_vals = (label_vals).astype(np.int32)
      
       # heldout_lp = np.mean(np.log(mean_probs[np.arange(mean_probs.shape[0]), label_vals]))
        
        
        #print(" ...Held_out nats: {:.3f}".format(heldout_lp))
        
       # Calculate validation accuracy

        for  step in range(20):

          sess.run(validation_accuracy_update_op, feed_dict={handle: heldout_handle})

        #valid_value = sess.run(valid_accuracy, feed_dict={handle: heldout_handle})
        loss_val_value, valid_accuracy_value, precision_value, recall_value, fpr_value, tpr_value,f1_value = sess.run([elbo_loss, valid_accuracy, precision, recall, fpr, tpr,f1], feed_dict = {handle:heldout_handle})
        print("Step: {:>3d} elboloss: {:.3f} Validation Accuracy: {:.3f} Precision: {:.3f} Recall: {:.3f} fpr:{:.3f} tpr:{:.3f} f1:{:.3f} ".format(
            step, loss_value, accuracy_value, precision_value, recall_value, fpr_value, tpr_value, f1_value))
       # print("Step: {:>3d} fpr: {:.3f} tpr: {:.3f} ".format(
           # step, fpr_value, tpr_value))
        #loss_value, accuracy_value, precision_value, recall_value, fpr_value, tpr_value = sess.run([elbo_loss, accuracy, precision, recall, fpr, tpr], feed_dict = {handle: train_handle})
        
        #print( Validation Accuracy: {:.3f}".format(valid_value))
        
        sess.run(reset_valid_op)
if __name__ == "__main__": 
  tf.app.run()
  

In [0]:
!pip install -q tensorflow==2.0.0-alpha0

In [0]:
# create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(X_train)
# apply a one-hot transformation to each label for use in the neural network
dy_train = tf.data.Dataset.from_tensor_slices(y_train)
# zip the x and y training data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(500).repeat().batch(8)

In [0]:
# do the same operations for the validation set
dx_valid = tf.data.Dataset.from_tensor_slices(X_test)
dy_valid = tf.data.Dataset.from_tensor_slices(y_test)
valid_dataset = tf.data.Dataset.zip((dx_valid, dy_valid)).shuffle(500).repeat().batch(30)

In [0]:
# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
next_element = iterator.get_next()

In [0]:
# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

In [0]:
def nn_model(in_data):
    bn = tf.layers.batch_normalization(in_data)
    fc1 = tf.layers.dense(bn, 19)
    fc2 = tf.layers.dense(fc1, 16)
    fc2 = tf.layers.dense(fc2, 14)
    fc3 = tf.layers.dense(fc2, 2)
    return fc3

In [0]:
# create the neural network model
logits = nn_model(next_element[0])

In [0]:
# add the optimizer and loss
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))
optimizer = tf.train.AdamOptimizer(1e-7).minimize(loss)
# get accuracy
predictions = tf.argmax(logits, 1)
equality = tf.equal(prediction, tf.argmax(next_element[1], 1))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))


#TP, TN, FP, FN
TP = tf.count_nonzero(predictions * y_test, dtype=tf.float32)
TN = tf.count_nonzero((predictions-1)*(y_test-1), dtype=tf.float32)
FP = tf.count_nonzero(predictions*(y_test-1), dtype=tf.float32)
FN = tf.count_nonzero((predictions-1)*y_test, dtype=tf.float32)
#precision, recall, f1
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * precision * recall / (precision + recall)
tpr = TP/(TP+FN)
fpr = FP/(TP+FN)






init_op = tf.global_variables_initializer()

In [38]:
# run the training
epochs = 600
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        if i % 50 == 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
    # now setup the validation run
    valid_iters = 1000
    # re-initialize the iterator, but this time with validation data
    sess.run(validation_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        
        loss_value, valid_accuracy, precision_value, recall_value, fpr_value, tpr_value,f1_value = sess.run([loss, accuracy, precision, recall, fpr, tpr,f1])
        print("Step: {:>3d} loss: {:.3f} Validation Accuracy: {:.3f} Precision: {:.3f} Recall: {:.3f} fpr:{:.3f} tpr:{:.3f} f1:{:.3f} ".format(
            i, loss_value, valid_accuracy, precision_value, recall_value, fpr_value, tpr_value, f1_value))
        
        
        
        
        acc = sess.run([accuracy])
        avg_acc += acc[0]
        print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters,(avg_acc ) ))          

Epoch: 0, loss: 42.896, training accuracy: 56.67%
Epoch: 50, loss: 48.911, training accuracy: 46.67%
Epoch: 100, loss: 40.610, training accuracy: 63.33%
Epoch: 150, loss: 40.221, training accuracy: 50.00%
Epoch: 200, loss: 41.872, training accuracy: 53.33%
Epoch: 250, loss: 48.391, training accuracy: 46.67%
Epoch: 300, loss: 50.877, training accuracy: 50.00%
Epoch: 350, loss: 49.859, training accuracy: 50.00%
Epoch: 400, loss: 35.234, training accuracy: 40.00%
Epoch: 450, loss: 41.353, training accuracy: 53.33%
Epoch: 500, loss: 40.190, training accuracy: 66.67%
Epoch: 550, loss: 43.613, training accuracy: 50.00%
Step:   0 loss: 46.889 Validation Accuracy: 0.533 Precision: 0.817 Recall: 0.700 fpr:0.157 tpr:0.700 f1:0.616 
Average validation set accuracy over 1000 iterations is 0.57%
Step:   1 loss: 49.155 Validation Accuracy: 0.500 Precision: 0.817 Recall: 0.500 fpr:0.112 tpr:0.500 f1:0.507 
Average validation set accuracy over 1000 iterations is 1.03%
Step:   2 loss: 46.049 Validation

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score,  f1_score

clf = svm.SVC(kernel='rbf',degree=3, probability=False,tol=0.001)
#clf = RandomForestClassifier(n_estimators=100,random_state=0)
#clf = LogisticRegression(C=0.1)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
fpr, tpr = roc_curve(y_test,y_pred)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))
print(fpr, tpr)

In [0]:

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state =1)

#reshape y-data to become column vector 
y_train = np.reshape(y_train, [-1,1])
y_test = np.reshape(y_test, [-1,1])

# Standardize the dataset 
scalar_x_train = StandardScaler().fit(X_train)
scalar_x_test = StandardScaler().fit(X_test)
X_train = scalar_x_train.transform(X_train)
X_test = scalar_x_test.transform(X_test)


# create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(X_train)
# apply a one-hot transformation to each label for use in the neural network
dy_train = tf.data.Dataset.from_tensor_slices(y_train)
# zip the x and y training data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(500).repeat().batch(30)



In [0]:
# do the same operations for the validation set
dx_valid = tf.data.Dataset.from_tensor_slices(X_test)
dy_valid = tf.data.Dataset.from_tensor_slices(y_test)
valid_dataset = tf.data.Dataset.zip((dx_valid, dy_valid)).shuffle(500).repeat().batch(30)
#valid_iter = valid_dataset.make_one_shot_iterator()
#valid_batch = valid_iter.get_next()

# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
next_element = iterator.get_next()

# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

In [0]:
#from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,  roc_curve
#Build the network 
model = tf.keras.Sequential([
   tf.keras.layers.Dense(16, input_dim=X_train.shape[1]),
   tf.keras.layers.Dense(16, activation=tf.nn.relu),
   tf.keras.layers.Dense(14, activation=tf.nn.relu),
   tf.keras.layers.Dense(10, activation=tf.nn.relu),
   tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])



model.summary()
#eager off
#opt = tf.train.AdamOptimizer(learning_rate=0.001)


model.compile( optimizer = tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy', metrics= ['accuracy'])

model.fit(train_dataset, steps_per_epoch=32, epochs=100,verbose=0 )

loss, accuracy = model.evaluate(valid_dataset, steps=32)


predictions = model.predict(X_test)
predictions = tf.cast(predictions, tf.float32)
TP = tf.count_nonzero(predictions * y_test, dtype=tf.float32)
EPOCHS = 100   
with tf.Session() as sess:
  for i in range(EPOCHS):
    _, loss_value,TP_value = sess.run([loss,TP])
    print(sess.run([loss,TP]))
    #print("iter: {:>3d},loss : {:.3f} accuracy : {:.3f}, TP: {:.3f}".format(i,loss_value, TP_value) )
  #print("loss:%f"% (loss))
  #print("accuracy: %f"%   (accuracy))
  #TP_value =sess.run(TP)
  #print("TP: %f"% ( TP_value))

"""# add the optimizer and loss
model.evaluate(valid_batch,valid_dataset)
predictions = model.predict(X_test)


TP, TN, FP, FN
TP = tf.count_nonzero(predictions * y_test, dtype=tf.float32)
TN = tf.count_nonzero((predictions-1)*(y_test-1), dtype=tf.float32)
FP = tf.count_nonzero(predictions*(y_test-1), dtype=tf.float32)
FN = tf.count_nonzero((predictions-1)*y_test, dtype=tf.float32)
#precision, recall, f1
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * precision * recall / (precision + recall)
tpr = TP/(TP+FN)
fpr = FP/(TP+FN)

  
  # run the training
epochs = 600

   
for step in range(epochs):
        #l, _, acc = sess.run([loss, optimizer, accuracy])
        if i % 50 == 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, accuracy * 100))
    # now setup the validation run
valid_iters = 100
    # re-initialize the iterator, but this time with validation data
  
    #avg_acc = 0
for step in range(valid_iters):
  #accuracy_value,precision_value, recall_value, tpr_value,fpr_value,f1_value = sess.run([accuracy,precision,recall,tpr,fpr,f1 ])
        #avg_acc += acc[0]
    #print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters,                  

print(" iterations : {:>3d}  Validation Accuracy: {:.3f} Precision: {:.3f} Recall: {:.3f} fpr:{:.3f} tpr:{:.3f} f1:{:.3f}".format(step,
             accuracy_value, precision_value, recall_value,fpr_value, tpr_value, f1_value))"""




